# The University of Melbourne, School of Computing and Information Systems
# COMP90049 Introduction to Machine Learning, 2020 Semester 1
-----
## Project 1: Understanding Student Success with Naive Bayes
-----
###### Student Name(s):
###### Python version:
###### Submission deadline: 11am, Wed 22 Apr 2019

This iPython notebook is a template which you will use for your Project 1 submission. 

Marking will be applied on the five functions that are defined in this notebook, and to your responses to the questions at the end of this notebook.

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find. 


# Question 1 is found below
## 1) Part A




## 1) Part B


In [222]:
import pandas as pd
path = "student.csv"

# This function should open a data file in csv, and transform it into a usable format 
def load_data(path):
    data = pd.read_csv(path)
    return data

rawData = load_data(path)
print(len(rawData))
print(rawData.head(3))

649
  school sex address famsize Pstatus  Medu  Fedu      Mjob      Fjob  reason  \
0     GP   M       U     GT3       T  high  high  services   teacher    home   
1     MS   M       R     GT3       T   mid   mid     other     other  course   
2     GP   F       R     LE3       T   mid   mid  services  services  course   

   ... internet romantic famrel freetime goout Dalc Walc health  \
0  ...      yes       no      5        2     3    1    2      5   
1  ...      yes       no      2        5     5    5    5      5   
2  ...      yes       no      3        3     2    2    2      3   

        absences Grade  
0    four_to_six     B  
1  more_than_ten     D  
2           none     D  

[3 rows x 30 columns]


In [284]:
from sklearn.model_selection import train_test_split

# This function should split a data set into a training set and hold-out test set
def split_data(rawData):
#     return rawData, rawData
    return train_test_split(rawData, test_size=0.5)

trainData, testData = split_data(rawData)
print(trainData.head(3))
print(testData.head(2))


    school sex address famsize Pstatus  Medu  Fedu   Mjob      Fjob reason  \
515     MS   M       U     GT3       T   mid   low  other  services   home   
231     GP   F       U     GT3       A  high  high  other   teacher   home   
67      GP   F       U     GT3       T   low   low  other     other   home   

     ... internet romantic famrel freetime goout Dalc Walc health  \
515  ...       no       no      3        2     3    1    3      4   
231  ...       no       no      4        1     4    1    1      1   
67   ...      yes       no      5        4     4    1    1      4   

         absences Grade  
515          none     D  
231  one_to_three     C  
67           none     C  

[3 rows x 30 columns]
    school sex address famsize Pstatus  Medu Fedu      Mjob   Fjob  \
197     GP   F       U     LE3       A  high  mid   teacher  other   
252     GP   M       U     LE3       T  high  mid  services  other   

         reason  ... internet romantic famrel freetime goout Dalc Walc h

In [287]:
import pandas as pd
featureFrames = None
CLASSES = ["A+", "A","B","C","D","F"]


def make_grade_struct():
    featureFrames[len(list(rawData))-1] = pd.DataFrame(columns=CLASSES, index=CLASSES)

def make_data_structs():
    global featureFrames
    featureFrames = []
    paramNames = []

    #For each feature
    for feature in list(rawData):
        
        #Find all the names of the paramaters by iterating through all data
        for j in range(len(rawData)):
            if str(rawData[feature][j]) not in paramNames:
                paramNames.append(str(rawData[feature][j]))        
        
        #Make a dataframe to store frequency information with format:     
        #     A+    A    B    C    D    F
        # T  NaN  NaN  NaN  NaN  NaN  NaN
        # A  NaN  NaN  NaN  NaN  NaN  NaN
        
        df = pd.DataFrame(columns=CLASSES, index=paramNames)
        featureFrames.append(df.copy())

        #Reset paramNames 
        paramNames = []
        
    #In case we don't see all the grade values, do this manually 
    make_grade_struct()    
    
def count_param_freq(data):
    #For every feature
    for i, feature in enumerate(list(data)):
        
        #Go and count the number of times each paramater resulted in what grade
        for j in range(len(data)):

            #Define the parameter feature, grade, and position in data struct
            param = str(data[feature].iloc[j])
            grade = data["Grade"].iloc[j]
            freqCount = featureFrames[i][grade][param]
        
            #Count the number of times that this parameter is seen
            if pd.isnull(freqCount):
                featureFrames[i][grade][param] = 1
            else:
                featureFrames[i][grade][param]+=1
                
                
#         print(feature)
#         print(featureFrames[i])
#         print()

            
            
 

# This function should build a supervised NB model
def train(data):
    make_data_structs()
    count_param_freq(data)
    return


train(trainData)

In [288]:
import operator
import random

eps = None
totTrainingRows = None


def find_probability(testGrade, testInst):

    #P(label|params) = const * p(param1|label) * p(param2|label) * ...
    probability = 1
    
    #For every parameter, find the probability and update main prob
    #for every feature name
    for i, label in enumerate(rawData):
        #Don't use the "Grade" column for the instance
        if i == len(list(rawData))-1:
            break
            

        # p(param1|label) = numberParam1|label / numberOfLabel
        #Get the actual paramater we're finding the probability of
        testParam = str(testInst[label])
        
        #Number of times this prameter was counted in training given the label
        try:
            #Try find the label, if you can't find it, or the value is null, probability *= eps
            paramFreq = featureFrames[i][testGrade][testParam]       
            if pd.isnull(paramFreq):
                probability*=eps
                continue
        except:
            probability*=eps
            continue

        
        #The number of times this grade has been counted in training
        gradeFreq = featureFrames[len(featureFrames)-1][testGrade][testGrade]            
        
#         print("Our instance where {}={}, according to main data=\n {}, \n has freq={}, with {} freq = {} with data= \n {}".format(label, testParam, featureFrames[i], paramFreq, testGrade, labelFreq, featureFrames[len(featureFrames)-1]))
        #Update probability
        probability *= (paramFreq/gradeFreq)

    probability *= gradeFreq/totTrainingRows
#     print(probability)
#     print()

    return probability

def predict_grade(instance):
    labelProbability = {"A+":0,"A":0,"B":0,"C":0,"D":0,"F":0}
    
    #For every potential label (A+, A, B, C, D, F), find P(label|params)
    for label in labelProbability.keys():
        labelProbability[label] = find_probability(label, instance)  
    
#     print("Calculated probabilities = {}".format(labelProbability))
    return max(labelProbability.items(), key=operator.itemgetter(1))[0]


# This function should predict the class for an instance or a set of instances, based on a trained model 
def predict(data):
    global eps
    global totTrainingRows
    eps = 1/len(data)
    totTrainingRows = len(data)    
    
    for i, inst in enumerate(data.iterrows()):
        print("instance number {} is predicted to be = {}, actual grade = {}".format(i, predict_grade(inst[1]), inst[1]["Grade"]))
    


predict(testData)
print(eps)

instance number 0 is predicted to be = B, actual grade = B
instance number 1 is predicted to be = B, actual grade = C
instance number 2 is predicted to be = D, actual grade = C
instance number 3 is predicted to be = B, actual grade = A
instance number 4 is predicted to be = D, actual grade = D
instance number 5 is predicted to be = D, actual grade = D
instance number 6 is predicted to be = D, actual grade = B
instance number 7 is predicted to be = D, actual grade = B
instance number 8 is predicted to be = D, actual grade = D
instance number 9 is predicted to be = C, actual grade = A
instance number 10 is predicted to be = B, actual grade = C
instance number 11 is predicted to be = D, actual grade = B
instance number 12 is predicted to be = F, actual grade = F
instance number 13 is predicted to be = C, actual grade = D
instance number 14 is predicted to be = D, actual grade = D
instance number 15 is predicted to be = D, actual grade = D
instance number 16 is predicted to be = D, actual 

instance number 149 is predicted to be = D, actual grade = F
instance number 150 is predicted to be = C, actual grade = B
instance number 151 is predicted to be = C, actual grade = C
instance number 152 is predicted to be = F, actual grade = F
instance number 153 is predicted to be = F, actual grade = D
instance number 154 is predicted to be = C, actual grade = B
instance number 155 is predicted to be = B, actual grade = D
instance number 156 is predicted to be = D, actual grade = F
instance number 157 is predicted to be = C, actual grade = D
instance number 158 is predicted to be = D, actual grade = D
instance number 159 is predicted to be = A+, actual grade = A
instance number 160 is predicted to be = D, actual grade = D
instance number 161 is predicted to be = C, actual grade = A
instance number 162 is predicted to be = D, actual grade = D
instance number 163 is predicted to be = C, actual grade = B
instance number 164 is predicted to be = A+, actual grade = C
instance number 165 is

instance number 288 is predicted to be = F, actual grade = D
instance number 289 is predicted to be = C, actual grade = D
instance number 290 is predicted to be = A, actual grade = B
instance number 291 is predicted to be = A+, actual grade = F
instance number 292 is predicted to be = D, actual grade = D
instance number 293 is predicted to be = A+, actual grade = D
instance number 294 is predicted to be = C, actual grade = D
instance number 295 is predicted to be = B, actual grade = C
instance number 296 is predicted to be = A, actual grade = B
instance number 297 is predicted to be = D, actual grade = B
instance number 298 is predicted to be = D, actual grade = F
instance number 299 is predicted to be = C, actual grade = C
instance number 300 is predicted to be = D, actual grade = D
instance number 301 is predicted to be = D, actual grade = F
instance number 302 is predicted to be = C, actual grade = A
instance number 303 is predicted to be = A, actual grade = B
instance number 304 is

In [301]:

accuracy = None

# This function should evaluate a set of predictions in terms of accuracy
def evaluate(data):
    global accuracy
    accuracy = {"success":0, "fail":0}
    for inst in data.iterrows():
#         print("instance number {} is predicted to be = {}, actual grade = {}".format(i, find_grade(inst[1]), inst[1]["Grade"]))
        if(predict_grade(inst[1]) == inst[1]["Grade"]):
            accuracy["success"] += 1
        else:
            accuracy["fail"] += 1
    s = accuracy["success"]
    f = accuracy["fail"]
    print("accuracy is {}. ({} successes and {} fails)".format((s/(s+f)), s, f))
            
    return (s/(s+f))

evaluate(testData)



accuracy is 0.3292307692307692. (107 successes and 218 fails)


0.3292307692307692

In [296]:
train(trainData)
evaluate(testData)

accuracy is 0.3292307692307692. (107 successes and 218 fails)


## 1) Part C
kjnsdfknjs

# Question 2

### Question 2: A Closer Look at Evaluation

- a You learnt in the lectures that precision, recall and f-1 measure can provide a more holistic and realistic picture of the classifier performance. (i) Explain the intuition behind accuracy, precision, recall, and F1-measure, (ii) contrast their utility, and (iii) discuss the difference between micro and macro averaging in the context of the data set. [no programming required]
- b Compute precision, recall and f-1 measure of your model’s predictions on the test data set (1) separately for each class, and (2) as a single number using macro-averaging. Compare the results against your accuracy scores from Question 1. In the context of the student dataset, and your response to question 2a analyze the additional knowledge you gained about your classifier performance.



#### a)
Accuracy - This is the proportion of guesses we got right compared to all guesses. More formally:

(True positives + True Negatives)/(True or Flase positives or negatives)


Precision - With respect to a class that we care to measure (maybe A+ students in this example), how often is our model correct. If we really care about one label more than the others, we want to see how accurately we can predict that class. More formally:

(True positives)/(True or False positives)


Recall - How good is our model at detecting our desired class. More formally:

(True positives)/(True positives + False Negatives)


F1 measure - A combined measure of precision and recall using the harmonic mean. This is important because precision and recall are usually inversely correlated (models with higher recall have lower precisino and vice versa). It is defined:

(2 * Precision*Recall)/(Precision + Recall)


#### b)


In [290]:
#First evaluate all the predicitons into TP, TN, FN and FP for each class

#Setup dataframes for storing the TP, TN, FN and FP results
labelAccuracy = {"A+":None,"A":None,"B":None,"C":None,"D":None,"F":None}

for label in labelAccuracy.keys():
    labelAccuracy[label] = pd.DataFrame(0, index=["positive", "negative"], columns=["true", "false"]) 
    
# print(labelAccuracy)


def add_one(label, truFal, posNeg):
    if pd.isnull(labelAccuracy[label][truFal][posNeg]):
        labelAccuracy[label][truFal][posNeg] = 1
    else:
        labelAccuracy[label][truFal][posNeg] += 1

        
def count_error_type(label, predictedGrade, actualGrade):
    if(predictedGrade == actualGrade and predictedGrade == label):
        add_one(label, "true", "positive")
    elif(label != predictedGrade and label != actualGrade):
        add_one(label, "true", "negative")
    elif(label == predictedGrade and label != actualGrade):
        add_one(label, "false", "positive")
    elif(label != predictedGrade and label == actualGrade):
        add_one(label, "false", "negative")
    

# This function should evaluate a set of predictions in terms of accuracy
def evaluateF1(data):
    #For each instance
    for inst in data.iterrows():
        
        predictedGrade = predict_grade(inst[1])
        actualGrade = inst[1]["Grade"]
        
        #Go through all the labels and find if it evaluated a TP, TN, FN or FP
        for label in labelAccuracy.keys():
            #If interesting key is correct, label as correct
            count_error_type(label, predictedGrade, actualGrade)
#             if(predictedGrade == "A+"):
#                 print("Our instance = {}, predicted grade = {}, label of interest = {}, labelTFPNDict = {}".format(actualGrade, predictedGrade, label, labelAccuracy))            
 
    return

eps = 1e-30

def safe_divide(num, denom):
    if (denom < eps):
        return 0
    else:
        return num/denom
    
def ave(lst, index): 
    sum = 0
    count = 0
    for indList in lst:
        sum += indList[index]
        count += 1
    return sum/count
        

def print_func():
    mac_ave = []
    for key in labelAccuracy.keys():
        theKey = key
        la = labelAccuracy
        
        precision = safe_divide(la[key]["true"]["positive"],(la[key]["true"]["positive"] + la[key]["false"]["positive"]))
        recall = safe_divide(la[key]["true"]["positive"],(la[key]["true"]["positive"] + la[key]["false"]["negative"]))
        f1 = safe_divide((2*precision*recall),(precision + recall))
        mac_ave.append([key, precision, recall, f1].copy())
      
        print("For the label {}, precision = {:.3f}, recall = {:.3f}, f-1 = {:.3f}".format(key, precision, recall, f1))
        print()
        
#     print("macro averaging scores: precision = {}, recall = {}, f1 = {}".format((mac_ave),(mac_ave),(mac_ave)))
    print("macro averaging scores: precision = {}, recall = {}, f1 = {}".format(ave(mac_ave, 1),ave(mac_ave,2),ave(mac_ave,3)))
        
 

evaluateF1(testData)
print(labelAccuracy)
print_func()

{'A+':           true  false
positive     0      8
negative   309      8, 'A':           true  false
positive     3     27
negative   266     29, 'B':           true  false
positive    11     38
negative   224     52, 'C':           true  false
positive    21     54
negative   203     47, 'D':           true  false
positive    59     76
negative   147     43, 'F':           true  false
positive    13     15
negative   258     39}
For the label A+, precision = 0.000, recall = 0.000, f-1 = 0.000

For the label A, precision = 0.100, recall = 0.094, f-1 = 0.097

For the label B, precision = 0.224, recall = 0.175, f-1 = 0.196

For the label C, precision = 0.280, recall = 0.309, f-1 = 0.294

For the label D, precision = 0.437, recall = 0.578, f-1 = 0.498

For the label F, precision = 0.464, recall = 0.250, f-1 = 0.325

macro averaging scores: precision = 0.2509687578735198, recall = 0.23426801276065987, f1 = 0.23496655900698368



### Question 3: Training Strategies 

There are other evaluation strategies, which tend to be preferred over the hold-out strategy you implemented in Question 1.
- a Select one such strategy, (i) describe how it works, and (ii) explain why it is preferable over hold-out evaluation. [no programming required]
- b Implement your chosen strategy from Question 3a, and report the accuracy score(s) of your classifier under this strategy. Compare your outcomes against your accuracy score in Question 1, and explain your observations in the context of your response to question 3a.


In [310]:
#part a)


n = 2
chunkSize = int(len(rawData)/n)
accuracyLog = []

for i in range(n):
    startIndex = i*chunkSize
    endIndex = startIndex + chunkSize
    
    #Deals with ugly divisors len(rawData)/chunkSize
    if i == n-1:
        endIndex = len(rawData)-1
    
#     tempDF = rawData.copy()
    
    tempTrainDF = rawData.drop(rawData.loc[startIndex:endIndex].index, inplace=False)
    tempTestDF = rawData.loc[startIndex:endIndex]
#     print(tempTestDF)
    #     print("indexes = {}, {}. Ogsize = {}, Size now = {}".format(startIndex, endIndex, len(rawData), len(tempDF)))
    
    print("Iteration number {}...".format(i+1))
    train(tempTrainDF)
    accuracyLog.append(evaluate(tempTestDF))
    print(featureFrames[0])
    print()
    

print("average value = {}".format((sum(accuracyLog)/len(accuracyLog))))



Iteration number 1...
accuracy is 0.3507692307692308. (114 successes and 211 fails)
   A+   A   B   C   D   F
GP  5  29  43  59  56  12
MS  5   7  13  20  41  34

Iteration number 2...
accuracy is 0.3353846153846154. (109 successes and 216 fails)
   A+   A   B   C   D   F
GP  5  20  44  62  67  20
MS  2   9  12  12  37  34

average value = 0.34307692307692306


## Questions (you may respond in a cell or cells below):

You should respond to Question 1 and two additional questions of your choice. A response to a question should take about 100–250 words, and make reference to the data wherever possible.

### Question 1: Naive Bayes Concepts and Implementation

- a Explain the ‘naive’ assumption underlying Naive Bayes. (1) Why is it necessary? (2) Why can it be problematic? Link your discussion to the features of the students data set. [no programming required]
- b Implement the required functions to load the student dataset, and estimate a Naive Bayes model. Evaluate the resulting classifier using the hold-out strategy, and measure its performance using accuracy.
- c What accuracy does your classifier achieve? Manually inspect a few instances for which your classifier made correct predictions, and some for which it predicted incorrectly, and discuss any patterns you can find.




### Question 4: Model Comparison

In order to understand whether a machine learning model is performing satisfactorily we typically compare its performance against alternative models. 
- a Choose one (simple) comparison model, explain (i) the workings of your chosen model, and (ii) why you chose this particular model. 
- b Implement your model of choice. How does the performance of the Naive Bayes classifier compare against your additional model? Explain your observations.

### Question 5: Bias and Fairness in Student Success Prediction

As machine learning practitioners, we should be aware of possible ethical considerations around the
applications we develop. The classifier you developed in this assignment could for example be used
to classify college applicants into admitted vs not-admitted – depending on their predicted
grade.
- a Discuss ethical problems which might arise in this application and lead to unfair treatment of the applicants. Link your discussion to the set of features provided in the students data set. [no programming required]
- b Select ethically problematic features from the data set and remove them from the data set. Use your own judgment (there is no right or wrong), and document your decisions. Train your Naive Bayes classifier on the resulting data set containing only ‘unproblematic’ features. How does the performance change in comparison to the full classifier?
- c The approach to fairness we have adopted is called “fairness through unawareness” – we simply deleted any questionable features from our data. Removing all problematic features does not guarantee a fair classifier. Can you think of reasons why removing problematic features is not enough? [no programming required]


In [181]:
print(featureFrames[22].index)

Index(['5', '2', '3', '4', '1'], dtype='object')


In [182]:
import pandas as pd
path = "student.csv"

# This function should open a data file in csv, and transform it into a usable format 
def load_data(path):
    data = pd.read_csv(path)
    return data

#make many df's 
#add a col to df live
dfList = []
rawData = load_data(path)
for col in rawData:
    dfList.append(Data)
    print(col)

NameError: name 'Data' is not defined